# Deep One Class Classification using Contradictions (DOC3)

# Imports

In [1]:
import os
import time
import torch
import logging
import numpy as np
import torch.nn as nn
from collections import OrderedDict
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    
print(device)

cuda


## 1. Helper Functions

### 1a. Evaluation (Loss, Eval Metrics)

In [2]:
# Loss Function
class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()
    
    def forward(self, ypred, ytrue, margin = 1.0, smooth = False):
        ypred = ypred.squeeze()
        if smooth:
            loss = torch.nn.Softplus()
            out = torch.mean(loss(margin - (ytrue * ypred)))
        else:
            out = torch.mean(torch.relu(margin - (ytrue * ypred)))
        return out

def f1score(y_true, scores):
    # Based on https://github.com/microsoft/EdgeML/blob/master/pytorch/edgeml_pytorch/trainer/drocc_trainer.py, line 147
    # Evaluation based on https://openreview.net/forum?id=BJJLHbb0-
    thresh = np.percentile(scores, 20)
    y_pred = np.where(scores >= thresh, 1, 0)
    
    prec, recall, f1_score, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary")
    return f1_score
   
    
def evalmetrics(y_true,scores, metric="f1"):    
    if metric == "f1":
        eval_metric = f1score(y_true, scores)
    else:
        eval_metric = roc_auc_score(y_true, scores)
    
    return eval_metric

### 1b. Data Set Loaders

In [3]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return torch.from_numpy(self.data[idx]), (self.labels[idx])

In [4]:
def load_data(path):
    train_data = np.load(os.path.join(path, 'train_data.npy'), allow_pickle = True)
    train_lab = np.ones((train_data.shape[0])) #All positive labelled data points collected
    test_data = np.load(os.path.join(path, 'test_data.npy'), allow_pickle = True)
    test_lab = np.load(os.path.join(path, 'test_labels.npy'), allow_pickle = True)
    
    num_features = train_data.shape[1]
    
    scaler = MinMaxScaler(feature_range=(-1., 1.))
    scaler.fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)

    train_samples = train_data.shape[0]
    test_samples = test_data.shape[0]
    print("Train Samples: ", train_samples)
    print("Test Samples: ", test_samples)
    
    return CustomDataset(train_data, train_lab), CustomDataset(test_data, test_lab), num_features

## 2. Models

In [5]:
class MLP(nn.Module):
    """
    Multi-layer perceptron with single hidden layer.
    """
    def __init__(self,
                 input_dim=2,
                 num_classes=1, 
                 num_hidden_nodes=128):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.num_hidden_nodes = num_hidden_nodes
        activ = nn.ReLU(True)
        self.feature_extractor = nn.Sequential(OrderedDict([
            ('fc', nn.Linear(self.input_dim, self.num_hidden_nodes)),
            ('relu1', activ)]))
        self.size_final = self.num_hidden_nodes

        self.classifier = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(self.size_final, self.num_classes))]))

    def forward(self, input):
        features = self.feature_extractor(input)
        logits = self.classifier(features.view(-1, self.size_final))
        return logits

## 3. Training Schedule

In [6]:
logging.basicConfig(filename='tabular_DOC3.log', level=logging.INFO)

In [7]:
# number of repetitions per experiment
# Results reported in paper are the summary statistics over 10 repetitions (i.e., n_reps = 10)
n_reps = 1

### 3.0 Training data: Arrhythmia

In [8]:
EPOCH = 300 
BATCH_SIZEU = 100
BATCH_SIZE = 100
METRIC = 'f1'

print('********************************')
print('Data: Arrhythmia')
logging.info('********************************')
logging.info('Data: Arrhythmia')
for exp in range(n_reps):
    %run process_odds.py -d data/Arrhythmia/arrhythmia -o data/Arrhythmia/
    for lam in [0.01]:
        for lr in [1e-3]:
            for Cu in [0.001]: 

                eps = 0.0
                dpos = 1.0-eps
                dneg = -(1.0+eps)

                print('================================')
                print('lam = {}, lr = {}, Cu = {}'.format(lam,lr,Cu))
                logging.info('================================')
                logging.info('lam = {}, lr = {}, Cu = {}'.format(lam,lr,Cu))



                print('Exp No: {}'.format(exp))
                logging.info('Exp No: {}'.format(exp))

                ###############################################################################################
                #                                     Train Data
                ###############################################################################################

                train_dataset, test_dataset, num_features = load_data("data/Arrhythmia")
                train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


                ###############################################################################################
                #                                     Evaluations on :-
                ###############################################################################################
                # Test Data.  Arrhythmia dataset has 132 test points
                test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=132, shuffle=False)

                # Train (used to Evaluate i.e. TP_rate etc.)
                train_loader_eval = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)


                net = MLP(input_dim=num_features)
                net = net.to(device)

                optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.99)
    
                train_loss = HingeLoss()
                unlabeled_posloss = HingeLoss()
                unlabeled_negloss = HingeLoss()

                for epoch in range(EPOCH+1):

                    loss_epoch = 0.0
                    n_batches = 0
                    epoch_start_time = time.time()

                    for data in train_loader:

                        # Training Data
                        inputs, labels = data
                        inputs = inputs.to(device).float()
                        ytr = np.ones((len(inputs),1))
                        ytr = torch.from_numpy(ytr).to(device).float()
                        outputs = net(inputs) 

                        # Random noise Universum
                        XU = torch.from_numpy(np.random.uniform(low=-1., high=1.0, size=(BATCH_SIZEU, num_features)))

                        XU = XU.to(device).float()
                        outputsU = net(XU)
                        yunppos = np.ones(BATCH_SIZEU)
                        yunpneg = -np.ones(BATCH_SIZEU)
                        yupos = torch.from_numpy(yunppos).to(device).float()
                        yuneg = torch.from_numpy(yunpneg).to(device).float()

                        # Zero the network parameter gradients
                        optimizer.zero_grad()

                        # Loss
                        loss = train_loss(outputs,ytr,smooth = False)

                        # Unlabeled data
                        lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
                        loss+=Cu*lossunlab

                        lam = torch.tensor(lam).to(device)
                        l2_reg = torch.tensor(0.).to(device)

                        for name, param in net.named_parameters():
                            l2_reg += torch.norm(param)**2

                        loss += lam * l2_reg

                        loss.backward()
                        optimizer.step()

                        loss_epoch += loss.item()
                        n_batches += 1


                    if epoch%50==0:

                        idx_label_score = []
                        net.eval()
                        with torch.no_grad():
                            for data in test_loader:
                                inputs, labels = data

                                labels = labels.cpu().data.numpy()

                                inputs = inputs.to(device).float()
                                outputs = net(inputs)
                                scores = outputs.data.cpu().numpy()
                                idx_label_score += list(zip(labels.tolist(),
                                                            scores.tolist()))

                        tstlabels, scores = zip(*idx_label_score)
                        tstlabels = np.array(tstlabels)

                        scores = np.array(scores)

                        Tst_score = evalmetrics(tstlabels,scores.flatten(), metric=METRIC)

                        # log epoch statistics
                        epoch_train_time = time.time() - epoch_start_time
                        logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test metric :{:.4f} '
                                    .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_score))

                        print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test metric :{:.4f} '
                                    .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_score))


                # Final Solution
                logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   Metric (Test) :{:.4f} '
                        .format(EPOCH, loss_epoch / n_batches, Tst_score))

                print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   Metric (Test) :{:.4f} '
                        .format(EPOCH, loss_epoch / n_batches, Tst_score))

********************************
Data: Arrhythmia
lam = 0.01, lr = 0.001, Cu = 0.001
Exp No: 0
Train Samples:  320
Test Samples:  132
  Epoch 0/300	 Time: 0.269	 Loss: 1.36924717 	 Test metric :0.7018 
  Epoch 50/300	 Time: 0.016	 Loss: 0.38464647 	 Test metric :0.7368 
  Epoch 100/300	 Time: 0.019	 Loss: 0.18516077 	 Test metric :0.7368 
  Epoch 150/300	 Time: 0.017	 Loss: 0.07043993 	 Test metric :0.7368 
  Epoch 200/300	 Time: 0.019	 Loss: 0.02529830 	 Test metric :0.7368 
  Epoch 250/300	 Time: 0.018	 Loss: 0.00927272 	 Test metric :0.7368 
  Epoch 300/300	 Time: 0.017	 Loss: 0.00465991 	 Test metric :0.7368 
 Final (TOT. EPOCHS 300)::  Loss: 0.00465991 	   Metric (Test) :0.7368 


### 3.1 Training data: Thyroid

In [9]:
EPOCH = 500 
BATCH_SIZEU = 100
BATCH_SIZE = 100
METRIC = 'f1'

print('********************************')
print('Data: Thyroid')
logging.info('********************************')
logging.info('Data: Thyroid')
for exp in range(n_reps):
    %run process_odds.py -d data/Thyroid/annthyroid -o data/Thyroid/
    for lam in [1e-6,]:
        for lr in [1e-3]:
            for Cu in [5.]:

                eps = 0.0
                dpos = 1.0-eps
                dneg = -(1.0+eps)

                print('================================')
                print('lam = {}, lr = {}, Cu = {}'.format(lam,lr,Cu))
                logging.info('================================')
                logging.info('lam = {}, lr = {}, Cu = {}'.format(lam,lr,Cu))



                print('Exp No: {}'.format(exp))
                logging.info('Exp No: {}'.format(exp))

                ###############################################################################################
                #                                     Train Data
                ###############################################################################################

                train_dataset, test_dataset, num_features = load_data("data/Thyroid")
                train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


                ###############################################################################################
                #                                     Evaluations on :-
                ###############################################################################################
                # Test Data. Thyroid dataset has 1068 test points
                test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1068, shuffle=False)

                # Train (used to Evaluate i.e. TP_rate etc.)
                train_loader_eval = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)


                net = MLP(input_dim=num_features)
                net = net.to(device)

                optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.99)  

                train_loss = HingeLoss()
                unlabeled_posloss = HingeLoss()
                unlabeled_negloss = HingeLoss()

                for epoch in range(EPOCH+1):

                    loss_epoch = 0.0
                    n_batches = 0
                    epoch_start_time = time.time()

                    for data in train_loader:

                        # Training Data
                        inputs, labels = data
                        inputs = inputs.to(device).float()
                        ytr = np.ones((len(inputs),1))
                        ytr = torch.from_numpy(ytr).to(device).float()
                        outputs = net(inputs) 

                        # Random noise Universum
                        XU = torch.from_numpy(np.random.uniform(low=-1., high=1.0, size=(BATCH_SIZEU, num_features)))

                        XU = XU.to(device).float()
                        outputsU = net(XU)
                        yunppos = np.ones(BATCH_SIZE)
                        yunpneg = -np.ones(BATCH_SIZE)
                        yupos = torch.from_numpy(yunppos).to(device).float()
                        yuneg = torch.from_numpy(yunpneg).to(device).float()


                        # Zero the network parameter gradients
                        optimizer.zero_grad()

                        # Loss
                        loss = train_loss(outputs,ytr,smooth = False)

                        # Unlabeled data
                        lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
                        loss+=Cu*lossunlab

                        lam = torch.tensor(lam).to(device)
                        l2_reg = torch.tensor(0.).to(device)

                        for name, param in net.named_parameters():
                            l2_reg += torch.norm(param)**2

                        loss += lam * l2_reg

                        loss.backward()
                        optimizer.step()

                        loss_epoch += loss.item()
                        n_batches += 1


                    if epoch%50==0:

                        idx_label_score = []
                        net.eval()
                        with torch.no_grad():
                            for data in test_loader:
                                inputs, labels = data

                                labels = labels.cpu().data.numpy()

                                inputs = inputs.to(device).float()
                                outputs = net(inputs)
                                scores = outputs.data.cpu().numpy()
                                idx_label_score += list(zip(labels.tolist(),
                                                            scores.tolist()))

                        tstlabels, scores = zip(*idx_label_score)
                        tstlabels = np.array(tstlabels)

                        scores = np.array(scores)
                        
                        Tst_score = evalmetrics(tstlabels,scores.flatten(), metric=METRIC)

                        # log epoch statistics
                        epoch_train_time = time.time() - epoch_start_time
                        logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test metric :{:.4f} '
                                    .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_score))

                        print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test metric :{:.4f} '
                                    .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_score))


                # Final Solution
                logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   Metric (Test) :{:.4f} '
                        .format(EPOCH, loss_epoch / n_batches, Tst_score))

                print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   Metric (Test) :{:.4f} '
                        .format(EPOCH, loss_epoch / n_batches, Tst_score))

********************************
Data: Thyroid
lam = 1e-06, lr = 0.001, Cu = 5.0
Exp No: 0
Train Samples:  6132
Test Samples:  1068
  Epoch 0/500	 Time: 0.211	 Loss: 3.10241471 	 Test metric :0.6484 
  Epoch 50/500	 Time: 0.213	 Loss: 0.14915057 	 Test metric :0.7334 
  Epoch 100/500	 Time: 0.215	 Loss: 0.11147142 	 Test metric :0.7378 
  Epoch 150/500	 Time: 0.212	 Loss: 0.06792299 	 Test metric :0.7507 
  Epoch 200/500	 Time: 0.208	 Loss: 0.09949921 	 Test metric :0.7450 
  Epoch 250/500	 Time: 0.215	 Loss: 0.10656645 	 Test metric :0.7464 
  Epoch 300/500	 Time: 0.211	 Loss: 0.05981821 	 Test metric :0.7550 
  Epoch 350/500	 Time: 0.207	 Loss: 0.03766230 	 Test metric :0.7550 
  Epoch 400/500	 Time: 0.203	 Loss: 0.15165700 	 Test metric :0.7579 
  Epoch 450/500	 Time: 0.208	 Loss: 0.21051489 	 Test metric :0.7579 
  Epoch 500/500	 Time: 0.205	 Loss: 0.10969081 	 Test metric :0.7579 
 Final (TOT. EPOCHS 500)::  Loss: 0.10969081 	   Metric (Test) :0.7579 


### 3.2 Training data: Abalone

In [10]:
EPOCH = 300 
BATCH_SIZEU = 100
BATCH_SIZE = 100
METRIC = 'f1'

print('********************************')
print('Data: Abalone')
logging.info('********************************')
logging.info('Data: Abalone')
for exp in range(n_reps):
    %run process_abalone.py -d data/Abalone/abalone.data -o data/Abalone/
    for lam in [0.1]:
        for lr in [1e-3]:
            for Cu in [0.01]:

                eps = 0.0
                dpos = 1.0-eps
                dneg = -(1.0+eps)

                print('================================')
                print('lam = {}, lr = {}, Cu = {}'.format(lam,lr,Cu))
                logging.info('================================')
                logging.info('lam = {}, lr = {}, Cu = {}'.format(lam,lr,Cu))


                print('Exp No: {}'.format(exp))
                logging.info('Exp No: {}'.format(exp))

                ###############################################################################################
                #                                     Train Data
                ###############################################################################################

                train_dataset, test_dataset, num_features = load_data("data/Abalone")
                train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


                ###############################################################################################
                #                                     Evaluations on :-
                ###############################################################################################
                # Test Data. Abalone dataset has 58 test points
                test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=58, shuffle=False)

                # Train (used to Evaluate i.e. TP_rate etc.)
                train_loader_eval = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)


                net = MLP(input_dim=num_features)
                net = net.to(device)

                optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.99)  

                train_loss = HingeLoss()
                unlabeled_posloss = HingeLoss()
                unlabeled_negloss = HingeLoss()

                for epoch in range(EPOCH+1):

                    loss_epoch = 0.0
                    n_batches = 0
                    epoch_start_time = time.time()

                    for data in train_loader:

                        # Training Data
                        inputs, labels = data
                        inputs = inputs.to(device).float()
                        ytr = np.ones((len(inputs),1))
                        ytr = torch.from_numpy(ytr).to(device).float()
                        outputs = net(inputs) 

                        # Random noise Universum
                        XU = torch.from_numpy(np.random.uniform(low=-1., high=1.0, size=(BATCH_SIZEU, num_features)))
        
                        XU = XU.to(device).float()
                        outputsU = net(XU)
                        yunppos = np.ones(BATCH_SIZE)
                        yunpneg = -np.ones(BATCH_SIZE)
                        yupos = torch.from_numpy(yunppos).to(device).float()
                        yuneg = torch.from_numpy(yunpneg).to(device).float()


                        # Zero the network parameter gradients
                        optimizer.zero_grad()

                        # Loss
                        loss = train_loss(outputs,ytr,smooth = False)

                        # Unlabeled data
                        lossunlab = unlabeled_posloss(outputsU,yupos,dpos,smooth = False) + unlabeled_negloss(outputsU,yuneg,dneg,smooth = False)
                        loss+=Cu*lossunlab

                        lam = torch.tensor(lam).to(device)
                        l2_reg = torch.tensor(0.).to(device)

                        for name, param in net.named_parameters():
                            l2_reg += torch.norm(param)**2

                        loss += lam * l2_reg

                        loss.backward()
                        optimizer.step()

                        loss_epoch += loss.item()
                        n_batches += 1


                    if epoch%50==0:

                        idx_label_score = []
                        net.eval()
                        with torch.no_grad():
                            for data in test_loader:
                                inputs, labels = data

                                labels = labels.cpu().data.numpy()

                                inputs = inputs.to(device).float()
                                outputs = net(inputs)
                                scores = outputs.data.cpu().numpy()
                                idx_label_score += list(zip(labels.tolist(),
                                                            scores.tolist()))

                        tstlabels, scores = zip(*idx_label_score)
                        tstlabels = np.array(tstlabels)

                        scores = np.array(scores)

                        Tst_score = evalmetrics(tstlabels,scores.flatten(), metric=METRIC)

                        # log epoch statistics
                        epoch_train_time = time.time() - epoch_start_time
                        logging.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test metric :{:.4f} '
                                    .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_score))

                        print('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f} \t Test metric :{:.4f} '
                                    .format(epoch , EPOCH, epoch_train_time, loss_epoch / n_batches, Tst_score))


                # Final Solution
                logging.info(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   Metric (Test) :{:.4f} '
                        .format(EPOCH, loss_epoch / n_batches, Tst_score))

                print(' Final (TOT. EPOCHS {})::  Loss: {:.8f} \t   Metric (Test) :{:.4f} '
                        .format(EPOCH, loss_epoch / n_batches, Tst_score))

********************************
Data: Abalone
lam = 0.1, lr = 0.001, Cu = 0.01
Exp No: 0
Train Samples:  1862
Test Samples:  58
  Epoch 0/300	 Time: 0.082	 Loss: 5.04095170 	 Test metric :0.7200 
  Epoch 50/300	 Time: 0.071	 Loss: 0.09643767 	 Test metric :0.7733 
  Epoch 100/300	 Time: 0.073	 Loss: 0.09609424 	 Test metric :0.7733 
  Epoch 150/300	 Time: 0.067	 Loss: 0.09611613 	 Test metric :0.7733 
  Epoch 200/300	 Time: 0.070	 Loss: 0.09612126 	 Test metric :0.7733 
  Epoch 250/300	 Time: 0.070	 Loss: 0.09610527 	 Test metric :0.7733 
  Epoch 300/300	 Time: 0.070	 Loss: 0.09614519 	 Test metric :0.7733 
 Final (TOT. EPOCHS 300)::  Loss: 0.09614519 	   Metric (Test) :0.7733 
